In [3]:
import os
import pandas as pd
import sklearn.metrics as s1
import sklearn.tree as s2
import sklearn.model_selection as s3 
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [4]:
a=r'C:\Users\Uttam Rajput\Downloads\Compressed\Manifold_Disease_Prediction_Using_Machine_Learning-main\Module_3\dataset\heart.csv'
data = pd.read_csv(a)
data[data.isnull().any(axis=1)].head()
train, test = s3.train_test_split(data, test_size=0.3)

In [5]:
data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [6]:
y_train = train['target']
del train['target']
x_train = train.copy()
y_test = test['target']
del test['target']
x_test = test.copy()

In [7]:
xtraining,xvalid,ytraining,yvalid = s3.train_test_split(x_train,y_train,test_size=0.5)

In [8]:
#specifying the initial learners
model1 = RandomForestClassifier(max_leaf_nodes=2, n_estimators=200,random_state=125)
model2 = LogisticRegression(max_iter=5290,solver='saga')
model3 = s2.DecisionTreeClassifier(max_leaf_nodes=6,min_samples_leaf=7,random_state=5,splitter="random")

#training the initial learners
model1.fit(xtraining.values,ytraining.values)
model2.fit(xtraining.values,ytraining.values)
model3.fit(xtraining.values,ytraining.values)

c:\Users\Uttam Rajput\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


DecisionTreeClassifier(max_leaf_nodes=6, min_samples_leaf=7, random_state=5,
                       splitter='random')

In [9]:
model1 = s2.DecisionTreeClassifier(max_leaf_nodes=5,min_samples_leaf=9,random_state=45,splitter="random")

model1.fit(xtraining.values,ytraining.values)
preds1 = model1.predict(xvalid.values)
predss1 = model1.predict(xtraining.values)

print(s1.accuracy_score(preds1,yvalid.values)*100)
print(s1.accuracy_score(predss1,ytraining.values)*100)

75.47169811320755
77.35849056603774


In [10]:
preds1 = model1.predict(xvalid.values)
preds2 = model2.predict(xvalid.values)
preds3 = model3.predict(xvalid.values)

predss1 = model1.predict(xtraining.values)
predss2 = model2.predict(xtraining.values)
predss3 = model3.predict(xtraining.values)

In [11]:
print(s1.accuracy_score(preds1,yvalid.values)*100)
print(s1.accuracy_score(preds2,yvalid.values)*100)
print(s1.accuracy_score(preds3,yvalid.values)*100)

print(s1.accuracy_score(predss1,ytraining.values)*100)
print(s1.accuracy_score(predss2,ytraining.values)*100)
print(s1.accuracy_score(predss3,ytraining.values)*100)

75.47169811320755
78.30188679245283
72.64150943396226
77.35849056603774
84.90566037735849
82.0754716981132


In [12]:

filename = 'model1.sav'
pickle.dump(model1, open(filename, 'wb'))

filename = 'model2.sav'
pickle.dump(model2, open(filename, 'wb'))

filename = 'model3.sav'
pickle.dump(model3, open(filename, 'wb'))


In [13]:
test_preds1 = model1.predict(x_test.values)
test_preds2 = model2.predict(x_test.values)
test_preds3 = model3.predict(x_test.values)

#making a new dataset for training our final model by stacking the predictions on the validation data
train_stack = np.column_stack((preds1,preds2,preds3))

#making the final test set for our final model by stacking the predictions on the test data
test_stack = np.column_stack((test_preds1,test_preds2,test_preds3))

In [14]:
final_model = RandomForestClassifier(max_leaf_nodes=50, n_estimators=195)

#training the final model on the stacked predictions
final_model.fit(train_stack,yvalid.values)

RandomForestClassifier(max_leaf_nodes=50, n_estimators=195)

In [15]:
final_predictions = final_model.predict(test_stack)
s1.accuracy_score(y_test,final_predictions)*100

81.31868131868131

In [16]:
cm = s1.confusion_matrix(y_test, final_predictions) 
print(cm)

[[22 10]
 [ 7 52]]


In [17]:
final_predictions = final_model.predict(train_stack)
s1.accuracy_score(yvalid,final_predictions)*100

81.13207547169812

In [18]:
filename = 'model_final.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [19]:
cm = s1.confusion_matrix(yvalid, final_predictions) 
print(cm)

[[36 13]
 [ 7 50]]
